## NAIS

In [1]:
import time
import datetime

In [2]:

def getTime(timeVal):
  t = datetime.datetime.strptime(timeVal, '%Y-%m-%d')
  d = t.timetuple()
  return int(time.mktime(d))

获得时间节点


1.   00/4/25-00/9/1
2.   00/9/1-01/1/1
3.   01/1/1-01/5/1
4.   01/5/1-01/9/1
5.   01/9/1-02/1/1
6.   02/1/1-02/5/1
7.   02/5/1-02/9/1
8.   02-9/1 - 03/1/1
9.   03/1/1 - 03/2/28



In [3]:
numT = 9
c1 =  getTime('2000-9-1')
c2 = getTime('2001-1-1')
c3 = getTime('2001-5-1')
c4 = getTime('2001-9-1')
c5 = getTime('2002-1-1')
c6 = getTime('2002-5-1')
c7 = getTime('2002-9-1')
c8 = getTime('2003-1-1')
c9 = getTime('2003-3-1')

In [4]:
numT = 31
c1 =  getTime('2000-9-1')
c2 =  getTime('2000-10-1')
c3=  getTime('2000-11-1')
c4 =  getTime('2000-12-1')
c5 = getTime('2001-1-1')
c6 =  getTime('2001-2-1')
c7 =  getTime('2001-3-1')
c8 =  getTime('2001-4-1')
c9 = getTime('2001-5-1')
c10 = getTime('2001-6-1')
c11 = getTime('2001-7-1')
c12 = getTime('2001-8-1')
c13 = getTime('2001-9-1')
c14 = getTime('2001-10-1')
c15 = getTime('2001-11-1')
c16 = getTime('2001-12-1')
c17 = getTime('2002-1-1')
c18 =  getTime('2002-2-1')
c19 =  getTime('2002-3-1')
c20 =  getTime('2002-4-1')
c21 = getTime('2002-5-1')
c22 = getTime('2002-6-1')
c23 = getTime('2002-7-1')
c24 = getTime('2002-8-1')
c25 = getTime('2002-9-1')
c26 = getTime('2002-10-1')
c27 = getTime('2002-11-1')
c28 = getTime('2002-12-1')
c29 = getTime('2003-1-1')
c30 = getTime('2003-2-1')
c31 = getTime('2003-3-1')

In [5]:
def getDate(timestamp):
  timeArray = time.localtime(timestamp)
  timestr = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
  date = datetime.datetime.strptime(timestr, "%Y-%m-%d %H:%M:%S")
  return date

In [6]:
import time
import math
import re

In [7]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

In [7]:
import tensorflow as tf

In [8]:
train_data = pd.read_csv('Data/ml-1m.train.rating', header=None, names=['userId', 'movieId', 'rating', 'timestamp'],sep='\t')

In [9]:
test_data = pd.read_csv('Data/ml-1m.test.rating', header=None, names=['userId', 'movieId', 'rating', 'timestamp'],sep='\t', engine='python')

In [10]:
test_neg_dict = {}
with open('Data/ml-1m.test.negative','r') as f:
    for line in f.readlines():
        test_neg_data = [int(i) for i in line.split('\t')[1:]]
        b = re.findall(r'[(](.*?),(.*?)[)]',  line.split('\t')[0])
        index_tuple = (int(b[0][0]), int(b[0][1]))
        test_neg_dict[index_tuple] = test_neg_data


划分 测试集训练集 --leave-one-out

保证每个用户的最后一个rating作为测试集

负样本采样，对于热门商品，为每个用户采样99个负样本

In [11]:
train_data['label'] = np.array([1]*len(train_data))

In [12]:
num_items = max(train_data.movieId.unique())+1
num_items

3706

In [13]:
def generate_item_time_map_dict(data):
    
  grouped = data.groupby(['movieId']).mean()
  return dict(zip(grouped.index, pd.cut(grouped['timestamp'].values, [0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18, \
                                                                     c19,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29,c30,c31], \
                                        labels=list(range(31)))))

ITmap = dict(generate_item_time_map_dict(train_data[['movieId','timestamp']]))

In [14]:
train_data['timestamp']= pd.cut(train_data.timestamp, [0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18, \
                                                                     c19,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29,c30,c31], \
                                        labels=list(range(31)))
test_data['timestamp']= pd.cut(test_data.timestamp, [0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18, \
                                                                     c19,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29,c30,c31], \
                                        labels=list(range(31)))

In [44]:
def generate_negative_samples(uindex, data, neg_count = 4):
    history = list(data[data.userId==uindex].movieId.values)
    negative_items = []
    for tmp in range(neg_count):
        j = np.random.choice(data.movieId.unique())
        while j in history:
            j = np.random.choice(data.movieId.unique())
        negative_items.append((j , 0 , ITmap[j]))
        
    return negative_items

In [15]:
def generate_train_list(data):
    train_list = []
    for u in data.userId.unique():
        train_list.append([(i[0], i[1])for i in data[data.userId==u][['movieId','timestamp']].values])
    return train_list

In [16]:
train_list = generate_train_list(train_data)
  

In [ ]:
test_data['label'] = np.array([0]*len(test_data))

In [17]:
test_list = generate_train_list(test_data)

In [18]:
for i in range(num_items):
    if i not in ITmap.keys():
        ITmap[i] = numT

In [19]:
def generate_test_all(test_list):
    test_all = []
    for ind, element in enumerate(test_list):
        test_all_u = []
        index = (ind, element[0][0])
        neg_items = test_neg_dict[index]
        for i in neg_items:
            test_all_u.append((i,ITmap[i]))
        test_all_u.append(element[0])
        test_all.append(test_all_u)
    return test_all

test_all = generate_test_all(test_list)

In [47]:
neg_count = 4
train_all = [ ]
for u in range(len(train_list)):
    u_train_all = []
    history = list(train_data[train_data.userId==u].movieId.values)
    for i in train_list[u]:
        u_train_all.append(i)
        for tmp in range(neg_count):
            j = np.random.choice(train_data.movieId.unique())
            while j in history:
                j = np.random.choice(train_data.movieId.unique())
            u_train_all.append((j , 0 , ITmap[j]))
    train_all.append(u_train_all)
    

In [20]:
import json
def default(o):
    if isinstance(o, np.int64): return int(o)  
    raise TypeError
with open('test_u' ,'a') as f:
    for line in test_all:
        linestr = json.dumps(line, default=default)
        f.write(linestr)
        f.write('\n')

In [21]:
with open('test_u' ,'a') as f:
    for line in train_list:
        linestr = json.dumps(line, default=default)
        f.write(linestr)
        f.write('\n')

In [22]:
ITmap = dict([(str(x), y) for x,y in ITmap.items()])

In [23]:
with open('itmap', 'w') as f:
    json.dump(ITmap, f, default=default)

In [23]:
embedding_size =16
weight_size = 16
verbose = 20
epoches = 200
beta = 0.5
alpha = 0
regs = [1e-5, 1e-7, 1e-4]
#regs = [0, 0, 0]
lambda_bilinear = regs[0]
gamma_bilinear = regs[1]
eta_bilinear = regs[2]
lr = 0.01
K = 10

In [24]:
def generate_train_data(train_list):
    u_batches = []
    ut_batches = []
    i_batches = []
    t_batches = []
    num_batches = []
    label_batches = []
    for u in range(len(train_list)):
        u_data = []
        u_tdata = []
        i_data=[]
        t_data = []
        num_data = []
        labels = []
        count =0
        for index in range(len(train_all[u])):
            u_hdata = train_list[u].copy()
            removed =0
            try: 
                u_hdata.remove(u_hdata[index])
                removed = 1
            except Exception:
                pass
            items, label, times = zip(*u_hdata)
            if removed:
                items = list(items)
                times = list(times)
                items.append(num_items)
                times.append(numT)
            num = len(train_list[u])
            u_data.append(items)
            u_tdata.append(times)
            i_data.append(train_all[u][index][0])
            t_data.append(train_all[u][index][2])
            labels.append(train_all[u][index][1])
            num_data.append(num)
        u_batches.append(np.array(u_data))
        ut_batches.append(np.array(u_tdata))
        i_batches.append(i_data)
        t_batches.append(t_data)
        num_batches.append(num_data)
        label_batches.append(labels)

    return u_batches, ut_batches, i_batches, t_batches, label_batches, num_batches
      
      
      

In [25]:
u_batches, ut_batches, i_batches, t_batches, label_batches, num_batches = generate_train_data(train_list)

In [27]:

def generate_test_data(test_list):
    u_batches = []
    ut_batches = []
    i_batches = []
    t_batches = []
    num_batches = []
    label_batches = []
    for u in range(len(test_list)):
        u_data = []
        u_tdata = []
        i_data=[]
        t_data = []
        num_data = []
        labels = []
        for index in range(len(test_all[u])):
            u_hdata = train_list[u].copy()
            items, label, times = zip(*u_hdata)
            num = len(train_list[u])
            u_data.append(items)
            u_tdata.append(times)
            i_data.append(test_all[u][index][0])
            t_data.append(test_all[u][index][2])
            labels.append(test_all[u][index][1])
            num_data.append(num)
        u_batches.append(np.array(u_data))
        ut_batches.append(np.array(u_tdata))
        i_batches.append(i_data)
        t_batches.append(t_data)
        num_batches.append(num_data)
        label_batches.append(labels)
    return u_batches, ut_batches, i_batches, t_batches, label_batches, num_batches

In [28]:
u_test_batches, ut_test_batches, i_test_batches, t_test_batches, label_test_batches, num_test_batches = \
generate_test_data(test_list)


### 定义参数

In [29]:
def get_inputs():
    with tf.name_scope('input_data'):
        user_input  = tf.placeholder(tf.int32, shape = [None, None], name='user_input')
        num_idx = tf.placeholder(tf.float32, shape=[None,1], name='num_idx')
        item_input = tf.placeholder(tf.int32, shape=[None,1], name='item_input')
        time_input = tf.placeholder(tf.int32, shape=[None, None], name='time_input')
        otime_input = tf.placeholder(tf.int32, shape=[None, 1], name='otime_input')
        labels = tf.placeholder(tf.float32, shape=[None,1], name='labels')
        return user_input, num_idx, item_input, time_input, otime_input, labels

In [30]:

with tf.name_scope('embeddings'):
    c1 = tf.Variable(tf.truncated_normal(shape=[num_items, embedding_size], mean=0.0, stddev=0.01),\
                    name='c1', dtype = tf.float32)
    c2 = tf.constant(0.0, tf.float32, [1, embedding_size], name='c2')
    embedding_Q_ = tf.concat([c1, c2], 0 , name='emebedding_Q_') 
    embedding_Q = tf.Variable(tf.truncated_normal(shape=[num_items,embedding_size]), name='embediing_Q', \
                             dtype=tf.float32)
    t1 = tf.Variable(tf.truncated_normal(shape=[numT+1, embedding_size], mean=0.0, stddev=0.01),\
                    name='t1', dtype = tf.float32)
    t2 = tf.constant(0.0, tf.float32, [1, embedding_size], name='t2')
    embedding_T = tf.concat([t1, t2], 0 , name='emebedding_T')
    bias = tf.Variable(tf.zeros(num_items), name='bias')
    
    #attention network variables
    W = tf.Variable(tf.truncated_normal(shape=[2*embedding_size, weight_size], mean=0.0, \
                    stddev=tf.sqrt(tf.div(2.0, weight_size + embedding_size))),name='Weights_for_MLP', dtype=tf.float32, trainable=True)
    bias_b = tf.Variable(tf.truncated_normal(shape=[1, weight_size], mean=0.0, \
        stddev=tf.sqrt(tf.divide(2.0, weight_size + embedding_size))),name='Bias_for_MLP', dtype=tf.float32, trainable=True)
    h = tf.Variable(tf.ones([weight_size, 1]), name='H_for_MLP', dtype=tf.float32)

In [31]:
user_input, num_idx, item_input, time_input, otime_input, labels = get_inputs()

In [32]:
def attention_MLP(q_, num_idx, B):#q_:(b,n,2e)
    with tf.name_scope("attention_MLP"):
        b = tf.shape(q_)[0]
        n = tf.shape(q_)[1]
        r = 2*embedding_size
        MLP_output = tf.matmul(tf.reshape(q_,[-1,r]), W) + bias_b #(b*n, e or 2*e) * (e or 2*e, w) + (1, w)
        MLP_output = tf.nn.dropout(MLP_output, 0.5)#(b*n, w)
#         fc_mean, fc_var = tf.nn.moments(MLP_output, axes=[0,1])
#         scale = tf.Variable(tf.ones([r]))
#         shift = tf.Variable(tf.zeros([r]))
#         epsilon = 0.001
#         MLP_output = tf.nn.batch_normalization(MLP_output, fc_mean, fc_var, \
#                                                shift, scale, epsilon)
        
        MLP_output = tf.nn.relu( MLP_output )
        #添加一个dropout
        A_ = tf.reshape(tf.matmul(MLP_output, h),[b,n]) #(b*n, w) * (w, 1) => (None, 1) => (b, n)
        # softmax for not mask features
        exp_A_ = tf.exp(A_)
        mask_mat = tf.sequence_mask(tf.reduce_sum(num_idx,1), maxlen = n, dtype = tf.float32) # (b, n)
        exp_A_ = mask_mat * exp_A_
        exp_sum = tf.reduce_sum(exp_A_,1, keepdims=True)  # (b, 1)
        exp_sum = tf.pow(exp_sum, tf.constant(beta, tf.float32, [1]))

        A = tf.expand_dims(tf.div(exp_A_, exp_sum),2) # (b, n, 1)
#         time_input = tf.expand_dims(time_input, 1)
#         time_input = tf.tile(time_input, [1, n, 1])
#         A = A + time_input

        #加入时间衰减
#         B = tf.expand_dims(B, 2)
#         A = A*B
      
        return tf.reduce_sum(A * embedding_q_, 1)      


In [33]:
with tf.name_scope("inference"):
    embedding_q_ = tf.nn.embedding_lookup(embedding_Q_, user_input) # (b, n, e)
    embedding_t_ = tf.nn.embedding_lookup(embedding_T, time_input)   #(b,n,e)
    embedding_q_ = tf.concat([embedding_q_, embedding_t_], axis=2) #(b,n,2e)
    
    
    embedding_q = tf.nn.embedding_lookup(embedding_Q, item_input) # (b, 1, e)
    embedding_t = tf.nn.embedding_lookup(embedding_T, otime_input) # (b, 1, e)
    embedding_q = tf.concat([embedding_q, embedding_t], axis=2)#(b,1,2e)
    embedding_p = attention_MLP(embedding_q_ * embedding_q, num_idx, time_input) #(b,2e)
    embedding_q = tf.reduce_sum(embedding_q, 1) #(b,2e)
    bias_i = tf.nn.embedding_lookup(bias, item_input)
#     coeff = tf.pow(num_idx, tf.constant(alpha, tf.float32, [1]))
#     output = tf.expand_dims(tf.reduce_sum(embedding_p*embedding_q, 1),1) + bias_i
#     output = tf.layers.dense(embedding_p*embedding_q, units=1, activation = None)
#     output = tf.map_fn(lambda x:x*5.0 , output)
#     output = tf.layers.dense(embedding_p*embedding_q, units=16, activation = tf.nn.sigmoid)
#     output = tf.layers.dense(output, 1, None)
    output = tf.sigmoid(tf.expand_dims(tf.reduce_sum(embedding_p*embedding_q, 1),1) + bias_i)

In [34]:
regs = [1e-5, 1e-7, 1e-4]
lr = 0.01
lambda_bilinear = regs[0]
gamma_bilinear = regs[1]
eta_bilinear = regs[2]
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.losses.log_loss(labels, output) + \
                lambda_bilinear * tf.reduce_sum(tf.abs(embedding_Q)) + \
                gamma_bilinear * tf.reduce_sum(tf.abs(embedding_Q_)) + \
                eta_bilinear * tf.reduce_sum(tf.abs(W)))
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdagradOptimizer(learning_rate=lr, initial_accumulator_value=1e-8).minimize(loss)

In [35]:
epoches =200

In [36]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_batch = len(u_batches)
    batch_index = range(num_batch)
    for epoch_count in range(epoches):
        #train
        train_begin = time.time()
        for index in batch_index:
            user_input_data = u_batches[index].astype(np.int32)
            num_idx_data= np.array(num_batches[index]).astype(np.float32)
            item_input_data= np.array(i_batches[index]).astype(np.int32)
            time_input_data = ut_batches[index].astype(np.float32)
            otime_input_data = np.array(t_batches[index])
            labels_data = np.array(label_batches[index]).astype(np.float32)
            feed_dict = {user_input: user_input_data, num_idx: num_idx_data[:, np.newaxis], item_input: item_input_data[:, np.newaxis],time_input:time_input_data,
                        otime_input:otime_input_data[:,np.newaxis], labels: labels_data[:, np.newaxis]}
            training_loss, _ = sess.run([loss, optimizer], feed_dict)
        train_time = time.time() - train_begin
        if epoch_count % verbose == 0:
            #train loss
            loss_begin = time.time()
            train_loss = 0.0
            for index in range(num_batch):
                user_input_data = u_batches[index].astype(np.int32)
                
                num_idx_data= np.array(num_batches[index]).astype(np.float32)
                item_input_data= np.array(i_batches[index]).astype(np.int32)
                time_input_data = ut_batches[index].astype(np.float32)
                otime_input_data = np.array(t_batches[index])
                
                labels_data = np.array(label_batches[index]).astype(np.float32)
                feed_dict = {user_input: user_input_data, num_idx: num_idx_data[:, np.newaxis], item_input: item_input_data[:, np.newaxis],time_input:time_input_data,
                            otime_input:otime_input_data[:,np.newaxis],labels: labels_data[:, np.newaxis]}
                train_loss += sess.run(loss, feed_dict)
            train_loss = train_loss/num_batch
            loss_time = time.time() - loss_begin
            #evaluate
            eval_begin = time.time() 
            hits, ndcgs, losses = [],[],[]
            for index in range(num_batch):
                user_input_data = u_test_batches[index].astype(np.int32)
                num_idx_data= np.array(num_test_batches[index]).astype(np.float32)
                item_input_data= np.array(i_test_batches[index]).astype(np.int32)
                time_input_data = ut_test_batches[index].astype(np.float32)
                otime_input_data = np.array(t_test_batches[index])
                
                labels_data = np.array(label_test_batches[index]).astype(np.float32)
                feed_dict = {user_input: user_input_data, num_idx: num_idx_data[:, np.newaxis], item_input: item_input_data[:, np.newaxis],time_input:time_input_data,
                        otime_input:otime_input_data[:,np.newaxis],labels: labels_data[:, np.newaxis]}
                predictions,test_loss = sess.run([output,loss], feed_dict = feed_dict)
                predictions = predictions.flatten()
#                 min_ = np.min(predictions)
#                 max_ = np.max(predictions)
#                 print(min_, max_)
#                 predictions = np.apply_along_axis(lambda x: 5*(x-min_)/(max_-min_), 0,predictions)

                neg_predict, pos_predict = predictions[:-1], predictions[-1]
                position = (neg_predict >= pos_predict).sum()
                hr = position < 10
                ndcg = math.log(2) / math.log(position+2) if hr else 0
                hits.append(hr)
                ndcgs.append(ndcg)  
                losses.append(test_loss)
            hr, ndcg, test_loss = np.array(hits).mean(), np.array(ndcgs).mean(), np.array(losses).mean()
            eval_time = time.time() - eval_begin
#             print("Epoch %d [%.1fs ]:  train_loss = %.4f" % (
#                     epoch_count,train_time, train_loss))    
            print("Epoch %d [ %.1fs]: HR = %.4f, NDCG = %.4f, loss = %.4f [%.1fs] train_loss = %.4f [%.1fs]" % (
                        epoch_count, train_time, hr, ndcg, test_loss, eval_time, train_loss, loss_time))

Epoch 0 [ 3.0s]: HR = 0.3000, NDCG = 0.1092, loss = 0.9850 [0.2s] train_loss = 0.9998 [0.6s]
Epoch 20 [ 2.3s]: HR = 0.3000, NDCG = 0.1869, loss = 0.6103 [0.1s] train_loss = 0.6957 [0.5s]
Epoch 40 [ 2.4s]: HR = 0.2800, NDCG = 0.1873, loss = 0.5844 [0.1s] train_loss = 0.6242 [0.6s]
Epoch 60 [ 2.2s]: HR = 0.3200, NDCG = 0.1926, loss = 0.5833 [0.2s] train_loss = 0.5724 [0.6s]
Epoch 80 [ 2.3s]: HR = 0.4000, NDCG = 0.2174, loss = 0.5891 [0.1s] train_loss = 0.5318 [0.6s]


ResourceExhaustedError: OOM when allocating tensor with shape[2985,597,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node optimizer/gradients/inference/attention_MLP/mul_1_grad/Mul}} = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](optimizer/gradients/inference/attention_MLP/Sum_2_grad/Tile, inference/concat)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node optimizer/gradients/embeddings/emebedding_Q__grad/GatherV2_2/_99}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_629_optimizer/gradients/embeddings/emebedding_Q__grad/GatherV2_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list o

In [ ]:
beta = 6
Epoch 0 [ 25.6s]: mae_mean = 1.0032, rmse_mean = 1.2028, loss = 1.8611 [3.1s] train_loss = 1.8357 [9.9s]
Epoch 20 [ 24.9s]: mae_mean = 0.8833, rmse_mean = 1.0809, loss = 1.4967 [3.0s] train_loss = 1.2582 [9.9s]
Epoch 40 [ 25.2s]: mae_mean = 0.8957, rmse_mean = 1.1006, loss = 1.5191 [3.0s] train_loss = 1.0992 [9.7s]

In [ ]:
Epoch 0 [ 25.9s]: mae_mean = 0.9289, rmse_mean = 1.1331, loss = 1.7034 [3.2s] train_loss = 1.6387 [10.3s]
Epoch 20 [ 25.5s]: mae_mean = 0.8303, rmse_mean = 1.0236, loss = 1.4031 [3.1s] train_loss = 1.1656 [10.1s]
Epoch 40 [ 26.5s]: mae_mean = 0.8180, rmse_mean = 1.0113, loss = 1.3615 [3.3s] train_loss = 1.0192 [10.7s]

In [ ]:
Epoch 0 [ 337.0s]: mae_mean = 0.9067, rmse_mean = 1.0797, loss = 1.5053 [36.5s] train_loss = 1.4938 [122.9s]
Epoch 20 [ 346.9s]: mae_mean = 0.8075, rmse_mean = 0.9704, loss = 1.2139 [38.8s] train_loss = 1.1453 [124.9s]
Epoch 40 [ 350.4s]: mae_mean = 0.8026, rmse_mean = 0.9657, loss = 1.1905 [36.7s] train_loss = 1.1035 [124.4s]
Epoch 60 [ 345.5s]: mae_mean = 0.8003, rmse_mean = 0.9639, loss = 1.1794 [36.8s] train_loss = 1.0815 [123.2s]
Epoch 80 [ 330.6s]: mae_mean = 0.7991, rmse_mean = 0.9636, loss = 1.1725 [37.6s] train_loss = 1.0639 [117.6s]
Epoch 100 [ 338.7s]: mae_mean = 0.7996, rmse_mean = 0.9639, loss = 1.1678 [38.0s] train_loss = 1.0501 [118.2s]